# Logistic Regression

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import itertools
import math
import os
import threading
import concurrent.futures
import numba as nb
import multiprocessing as mp

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import wait
from concurrent.futures import as_completed
from joblib import Parallel, delayed
from numba import njit
from numba import jit
from numba import cuda

/Users/alexherron/opt/miniconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load in data
df = pd.read_csv('cleaned_bullying.csv')

# Drop columns
df = df.drop(columns=['Unnamed: 0',
                      'Bullied_on_school_property_in_past_12_months',
                      'Bullied_not_on_school_property_in_past_12_months',
                      'Cyber_bullied_in_past_12_months'])

# Split into X and y
X = df.drop('bullied', axis=1).values
y = df['bullied'].values

# Load copy of dataframe
data = df

# Train/test split
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=13)

In [3]:
# Set values for learning rate, number of iterations, and batch size
learning_rate = 0.0001
number_iterations = 100_000
batch_num = 50
digits = 5

# 1. Logistic Regression from Scratch

In [4]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # Initialize the weights
        self.theta = np.zeros(X.shape[1])
        
        # Gradient descent
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient
            
            if i % 10000 == 0:
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
    
    def predict_proba(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold=0.5):
        return self.predict_proba(X) >= threshold
    
# Function for training model and making predictions
def run_logistic_regression():
    lr = LogisticRegression(lr=learning_rate,
                            num_iter=number_iterations)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    return y_pred

In [5]:
def evaluate():
    
    # Run and time model
    start = time.time()
    y_pred = run_logistic_regression()
    end = time.time()
    time_diff = round(end - start, digits)

    # Evaluate predictions
    auc = round(roc_auc_score(y_test, y_pred), digits)
    accuracy = round(accuracy_score(y_test, y_pred), digits)
    precision = round(precision_score(y_test, y_pred), digits)
    recall = round(recall_score(y_test, y_pred), digits)
    f1 = round(f1_score(y_test, y_pred), digits)

    # Print Results
    print(f'Time: {time_diff}')
    print(f'AUC: {auc}')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'AUC: {recall}')
    print(f'F1: {f1}')

In [6]:
evaluate()

Time: 57.3179
AUC: 0.62521
Accuracy: 0.67092
Precision: 0.63946
AUC: 0.39985
F1: 0.49203


In [7]:
%load_ext line_profiler

In [8]:
%lprun -f LogisticRegression.fit run_logistic_regression()

# 2. Optimize Using SGD

In [9]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, random_state=13, batch_size=5):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.random_state = random_state
        self.batch_size = batch_size
        
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
        
    # optimized random batch selection
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # Initialize the weights
        self.theta = np.zeros(X.shape[1])
        
        # Gradient descent
        rng = np.random.default_rng()
        idx = rng.integers(len(X), size=(self.num_iter, 10))
        for i in range(self.num_iter):
            batch_idx = idx[i]
            batch_X = X[batch_idx]
            batch_y = y[batch_idx]
            z = np.dot(batch_X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(batch_X.T, (h - batch_y)) / batch_y.size
            self.theta -= self.lr * gradient
            
            if i % 10000 == 0:
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
    
    def predict_proba(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold=0.5):
        return self.predict_proba(X) >= threshold
    
def run_logistic_regression():
    lr = LogisticRegression(lr=learning_rate,
                            num_iter=number_iterations,
                            batch_size=batch_num)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    return y_pred

In [10]:
evaluate()

Time: 1.68186
AUC: 0.6249
Accuracy: 0.67061
Precision: 0.63886
AUC: 0.39947
F1: 0.49157


In [11]:
%lprun -f LogisticRegression.fit run_logistic_regression()

# 3. Optimize By Normalizing Input Features

In [12]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, random_state=13, batch_size=5):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.random_state = random_state
        self.batch_size = batch_size
        
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __normalize(self, X):
        X_norm = X.copy()
        self.mean = np.mean(X_norm, axis=0)
        self.std = np.std(X_norm, axis=0)
        X_norm = (X_norm - self.mean) / (self.std + 1e-8)
        return X_norm
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    # optimized random batch selection
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # Normalize the input features
        X = self.__normalize(X)
        
        # Initialize the weights
        self.theta = np.zeros(X.shape[1])
        
        # Gradient descent
        rng = np.random.default_rng()
        idx = rng.integers(len(X), size=(self.num_iter, 10))
        for i in range(self.num_iter):
            batch_idx = idx[i]
            batch_X = X[batch_idx]
            batch_y = y[batch_idx]
            z = np.dot(batch_X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(batch_X.T, (h - batch_y)) / batch_y.size
            self.theta -= self.lr * gradient
            
            if i % 10000 == 0:
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
    
    def predict_proba(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold=0.5):
        return self.predict_proba(X) >= threshold

def run_logistic_regression():
    lr = LogisticRegression(lr=learning_rate,
                            num_iter=number_iterations,
                            batch_size=batch_num)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    return y_pred

In [13]:
evaluate()

Time: 1.84188
AUC: 0.62838
Accuracy: 0.6697
Precision: 0.6264
AUC: 0.4246
F1: 0.50613


In [14]:
%lprun -f LogisticRegression.fit run_logistic_regression()

# 4. Optimize with Threading

In [15]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, random_state=13, batch_size=5, num_threads=4):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.random_state = random_state
        self.batch_size = batch_size
        self.num_threads = num_threads
        
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
        
    def __batch_gradient_descent(self, batch_X, batch_y):
        z = np.dot(batch_X, self.theta)
        h = self.__sigmoid(z)
        gradient = np.dot(batch_X.T, (h - batch_y)) / batch_y.size
        self.theta -= self.lr * gradient
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)

        # Initialize the weights
        self.theta = np.zeros(X.shape[1])

        # Create thread pool
        with ThreadPoolExecutor(max_workers=self.num_threads) as executor:
            for i in range(self.num_iter):
                # Generate random batch indices
                batch_idx = np.random.randint(len(X) - self.batch_size + 1, size=self.num_threads)
                batch_idx = [idx + j for j in range(self.batch_size) for idx in batch_idx]

                # Split data into batches
                batches_X = np.split(X[batch_idx], self.num_threads)
                batches_y = np.split(y[batch_idx], self.num_threads)

                # Submit batches for processing
                futures = []
                for j in range(self.num_threads):
                    futures.append(executor.submit(self.__batch_gradient_descent, batches_X[j], batches_y[j]))

                # Wait for all worker threads to finish
                for future in futures:
                    future.result()

                if i % 10000 == 0:
                    z = np.dot(X, self.theta)
                    h = self.__sigmoid(z)
    
    def predict_proba(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold=0.5):
        return self.predict_proba(X) >= threshold

In [16]:
evaluate()

Time: 36.25018
AUC: 0.62579
Accuracy: 0.67122
Precision: 0.63939
AUC: 0.40175
F1: 0.49345


In [17]:
%lprun -f LogisticRegression.fit run_logistic_regression()

# 5. Optimize with Numba / CUDA

In [18]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, random_state=13, batch_size=5, verbose=True):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.random_state = random_state
        self.batch_size = batch_size
        self.verbose = verbose
        
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

    # GD using numba
    def fit_numba(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        self.theta = self.gradient_descent(X, y, np.zeros(X.shape[1]), self.lr, self.num_iter)
        
    @staticmethod    
    @jit(nopython=True)
    def gradient_descent(X, y, theta, lr, num_iter):
        for i in range(num_iter):
            z = np.dot(X, theta)
            h = 1/(1+np.exp(-z))
            gradient = np.dot(X.T, (h-y))/y.size
            theta -= lr*gradient
            
        return theta
    
    # computing mini-batch gradient
    @staticmethod    
    @jit(nopython=True)
    def SGD(X, y, theta, lr, num_iter, batch_size, idx):
        for i in range(num_iter):
            batch_idx = idx[i]
            batch_X = X[batch_idx]
            batch_y = y[batch_idx]
            z = np.dot(batch_X, theta)
            h = 1/(1+np.exp(-z))
            gradient = np.dot(batch_X.T, (h - batch_y)) / batch_y.size
            theta -= lr * gradient
            
        return theta

    # use cuda kernel to do matrix multiplication
    def fit_cuda(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # Initialize the weights
        TPB=16
        
        X_d = cuda.to_device(np.float32(X))
        y_d = cuda.to_device(y.reshape([y.shape[0],1]))
        z_d = cuda.to_device(np.zeros([X.shape[0],1], dtype=np.float32))
        h_d = cuda.to_device(np.zeros(z_d.shape, dtype=np.float32))
        
        self.theta = np.zeros(X.shape[1])
        theta_d = cuda.to_device(np.zeros((X.shape[1],1), dtype=np.float32))
        
        threadsperblock = (TPB, TPB)
        blockspergrid_x = math.ceil(z_h.shape[0] / threadsperblock[0])
        blockspergrid_y = math.ceil(z_h.shape[1] / threadsperblock[1])
        blockspergrid = (blockspergrid_x, blockspergrid_y)
        
        # Gradient descent
        for i in range(self.num_iter):
            #cuda_dot[blockspergrid, threadsperblock](X, theta_d, z_d)
            fast_matmul[blockspergrid, threadsperblock](X, theta_d, z_d)
            #cuda_sigmoid[blockspergrid, threadsperblock](z_d, h_d)
            #h = h_d.copy_to_host()
            z = z_d.copy_to_host()
            h = self.__sigmoid(z)
            
            gradient = np.dot(X.T, (h.reshape(y.shape) - y)) / y.size
            self.theta -= self.lr * gradient
            
            if self.verbose:
                if i % 10000 == 0:
                    z = np.dot(X, self.theta)
                    h = self.__sigmoid(z)
                    print(f'Step: {i}')
                    print(f'loss: {self.__loss(h, y)}')
                    
    def fit_optimized_numba(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # Initialize the weights
        self.theta = np.zeros(X.shape[1])
        
        # Gradient descent
        rng = np.random.default_rng()
        idx = rng.integers(len(X), size=(self.num_iter, 10))
        self.theta = self.SGD(X, y, np.zeros(X.shape[1]), self.lr, self.num_iter, self.batch_size, idx)
    

    def predict_proba(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold=0.5):
        return self.predict_proba(X) >= threshold

In [19]:
@cuda.jit
def cuda_dot(A, B, C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        C[i, j] = 0
        for k in range(A.shape[1]):
            C[i, j] += A[i, k] * B[k, j]
        C[i,j] += C[i,j]

@cuda.jit
def cuda_sigmoid(z, out):
    i, j = cuda.grid(2)
    if i < z.shape[0] and j < z.shape[1]:
        out[i, j] = 1 / (1 + math.exp(-z[i, j]))

@cuda.jit
def cuda_gradient(X, h, y, gradient):
    i = cuda.grid(1)
    if i < X.shape[1]:
        gradient[i] = 0
        for j in range(X.shape[0]):
            gradient[i] += (h[j, 0] - y[j]) * X[j, i]
        gradient[i] /= y.size

@cuda.jit
def cuda_subtract(a, b, diff):
    i, j = cuda.grid(2)
    if i < a.shape[0] and j < b.shape[1]:
        diff[i,j] = a[i,j] - b[i,j]
            
@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B using CUDA shared memory.

    Reference: https://stackoverflow.com/a/64198479/13697228 by @RobertCrovella
    """
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=np.float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=np.float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = np.float32(0.)
    for i in range(bpg):
        # Preload data into shared memory
        sA[ty, tx] = 0
        sB[ty, tx] = 0
        if y < A.shape[0] and (tx + i * TPB) < A.shape[1]:
            sA[ty, tx] = A[y, tx + i * TPB]
        if x < B.shape[1] and (ty + i * TPB) < B.shape[0]:
            sB[ty, tx] = B[ty + i * TPB, x]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[ty, j] * sB[j, tx]

        # Wait until all threads finish computing
        cuda.syncthreads()
    if y < C.shape[0] and x < C.shape[1]:
        C[y, x] = tmp

In [20]:
def run_logistic_regression(version):
    lr = LogisticRegression(lr=learning_rate,
                            num_iter=number_iterations,
                            batch_size=batch_num,
                            verbose=False)
    assert version in ['numba', 'optimized_numba', 'cuda']
    if version == 'numba':
        lr.fit_numba(X_train, y_train)
    elif version == 'optimized_numba':
        lr.fit_optimized_numba(X_train, y_train)
    y_pred = lr.predict(X_test)
    return y_pred

def evaluate(version):
    
    # Run and time model
    start = time.time()
    y_pred = run_logistic_regression(version)
    end = time.time()
    time_diff = round(end - start, digits)

    # Evaluate predictions
    auc = round(roc_auc_score(y_test, y_pred), digits)
    accuracy = round(accuracy_score(y_test, y_pred), digits)
    precision = round(precision_score(y_test, y_pred), digits)
    recall = round(recall_score(y_test, y_pred), digits)
    f1 = round(f1_score(y_test, y_pred), digits)

    # Print Results
    print(f'Time: {time_diff}')
    print(f'AUC: {auc}')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'AUC: {recall}')
    print(f'F1: {f1}')

In [21]:
evaluate('numba')

Time: 45.98526
AUC: 0.62521
Accuracy: 0.67092
Precision: 0.63946
AUC: 0.39985
F1: 0.49203


In [22]:
evaluate('optimized_numba')

Time: 0.79876
AUC: 0.62407
Accuracy: 0.67046
Precision: 0.64035
AUC: 0.39528
F1: 0.48882


In [23]:
# Note: need CUDA GPU to run cuda model
#evalute('cuda')